In [ ]:
#bi_lstm_model.py




# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf


class Model(object):

    def __init__(self,
                 nh1,
                 nh2,
                 ny,
                 nz,
                 de,
                 cs,
                 lr,
                 lr_decay,
                 embedding,
                 max_gradient_norm,
                 model_cell='rnn',
                 model='basic_model',
                 nonstatic=False):

        self.batch_size = tf.placeholder(dtype=tf.int32, shape=None)
        self.input_x=tf.placeholder(tf.int32,shape=[None,None,cs],name='input_x')
        self.input_y=tf.placeholder(tf.int32,shape=[None,None],name="input_y")
        self.input_z=tf.placeholder(tf.int32,shape=[None,None],name='input_z')
        self.keep_prob=tf.placeholder(dtype=tf.float32,name='keep_prob')

        self.lr=tf.Variable(lr,dtype=tf.float32)

        self.learning_rate_decay_op = self.lr.assign(
            self.lr * lr_decay)


        #Creating embedding input
        with tf.device("/cpu:0"),tf.name_scope('embedding'):
            if nonstatic:
                W=tf.constant(embedding,name='embW',dtype=tf.float32)
            else:
                W=tf.Variable(embedding,name='embW',dtype=tf.float32)
            inputs=tf.nn.embedding_lookup(W,self.input_x)
            inputs=tf.reshape(inputs,[self.batch_size,-1,cs*de])

        #Droupout embedding input
        inputs=tf.nn.dropout(inputs,keep_prob=self.keep_prob,name='drop_inputs')

        #Create the internal multi-layer cell for rnn
        if model_cell=='rnn':
            single_cell0=tf.nn.rnn_cell.BasicRNNCell(nh1)
            single_cell1=tf.nn.rnn_cell.BasicRNNCell(nh1)
            single_cell2=tf.nn.rnn_cell.BasicRNNCell(nh2)
        elif model_cell=='lstm':
            single_cell0=tf.nn.rnn_cell.BasicLSTMCell(nh1,state_is_tuple=True)
            single_cell1=tf.nn.rnn_cell.BasicLSTMCell(nh1,state_is_tuple=True)
            single_cell2=tf.nn.rnn_cell.BasicLSTMCell(nh2,state_is_tuple=True)
        elif model_cell=='gru':
            single_cell0=tf.nn.rnn_cell.GRUCell(nh1)
            single_cell1=tf.nn.rnn_cell.GRUCell(nh1)
            single_cell2=tf.nn.rnn_cell.GRUCell(nh2)
        else:
            raise 'model_cell error!'
        #DropoutWrapper rnn_cell
        single_cell0 = tf.nn.rnn_cell.DropoutWrapper(single_cell0,output_keep_prob=self.keep_prob)
        single_cell1 = tf.nn.rnn_cell.DropoutWrapper(single_cell1, output_keep_prob=self.keep_prob)
        single_cell2 = tf.nn.rnn_cell.DropoutWrapper(single_cell2, output_keep_prob=self.keep_prob)
        
        self.init_state=single_cell1.zero_state(self.batch_size,dtype=tf.float32) 
        
        
        #Bi-RNN1

        x_len = tf.cast(tf.shape(inputs)[1], tf.int64)
        batch=2
        with tf.variable_scope('bi_rnn1'):
            self.outputs1,self.state1=tf.nn.bidirectional_dynamic_rnn(
                single_cell0,
                single_cell1,
                inputs,
                sequence_length=[x_len]*batch,
                dtype=tf.float32
            )

        self.outputs1=tf.concat(2,self.outputs1)

        
        #RNN2
        with tf.variable_scope('rnn2'):
            self.outputs2,self.state2=tf.nn.dynamic_rnn(
                cell=single_cell2,
                inputs=self.outputs1,
                initial_state=self.init_state,
                dtype=tf.float32
            )

        #outputs_y
        with tf.variable_scope('output_sy'):
            w_y=tf.get_variable("softmax_w_y",[2*nh1,ny])
            b_y=tf.get_variable("softmax_b_y",[ny])
            outputs1=tf.reshape(self.outputs1,[-1,2*nh1])
            sy=tf.nn.xw_plus_b(outputs1,w_y,b_y)
            self.sy_pred = tf.reshape(tf.argmax(sy, 1), [self.batch_size, -1])
        #outputs_z
        with tf.variable_scope('output_sz'):
            w_z = tf.get_variable("softmax_w_z", [nh2, nz])
            b_z = tf.get_variable("softmax_b_z", [nz])
            outputs2 = tf.reshape(self.outputs2, [-1, nh2])
            sz = tf.nn.xw_plus_b(outputs2, w_z,b_z)
            self.sz_pred = tf.reshape(tf.argmax(sz, 1), [self.batch_size, -1])
        #loss
        with tf.variable_scope('loss'):
            label_y = tf.reshape(self.input_y, [-1])
            loss1 = tf.nn.sparse_softmax_cross_entropy_with_logits(sy, label_y)
            label_z = tf.reshape(self.input_z, [-1])
            loss2 = tf.nn.sparse_softmax_cross_entropy_with_logits(sz, label_z)
            self.loss=tf.reduce_sum(0.5*loss1+0.5*loss2)/tf.cast(self.batch_size,tf.float32)

        tvars=tf.trainable_variables()
        grads,_=tf.clip_by_global_norm(tf.gradients(self.loss,tvars),max_gradient_norm)
        optimizer=tf.train.GradientDescentOptimizer(self.lr)
        self.train_op=optimizer.apply_gradients(zip(grads,tvars))

    def cost(output, target):
        # Compute cross entropy for each frame.
        cross_entropy = target * tf.log(output)
        cross_entropy = -tf.reduce_sum(cross_entropy, reduction_indices=2)
        mask = tf.sign(tf.reduce_max(tf.abs(target), reduction_indices=2))
        cross_entropy *= mask
        # Average over actual sequence lengths.
        cross_entropy = tf.reduce_sum(cross_entropy, reduction_indices=1)
        cross_entropy /= tf.reduce_sum(mask, reduction_indices=1)
        return tf.reduce_mean(cross_entropy)

In [ ]:
#model.py


# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf

class Model(object):

    def __init__(self,
                 nh1,
                 nh2,
                 ny,
                 nz,
                 de,
                 cs,
                 lr,
                 lr_decay,
                 embedding,
                 max_gradient_norm,
                 model_cell='rnn',
                 model='basic_model',
                 nonstatic=False):

        self.batch_size = tf.placeholder(dtype=tf.int32, shape=None)
        self.input_x=tf.placeholder(tf.int32,shape=[None,None,cs],name='input_x')
        self.input_y=tf.placeholder(tf.int32,shape=[None,None],name="input_y")
        self.input_z=tf.placeholder(tf.int32,shape=[None,None],name='input_z')
        self.keep_prob=tf.placeholder(dtype=tf.float32,name='keep_prob')

        self.lr=tf.Variable(lr,dtype=tf.float32)

        self.learning_rate_decay_op = self.lr.assign(
            self.lr * lr_decay)


        #Creating embedding input
        with tf.device("/cpu:0"),tf.name_scope('embedding'):
            if nonstatic:
                W=tf.constant(embedding,name='embW',dtype=tf.float32)
            else:
                W=tf.Variable(embedding,name='embW',dtype=tf.float32)
            inputs=tf.nn.embedding_lookup(W,self.input_x)
            inputs=tf.reshape(inputs,[self.batch_size,-1,cs*de])

        #Droupout embedding input
        inputs=tf.nn.dropout(inputs,keep_prob=self.keep_prob,name='drop_inputs')

        #Create the internal multi-layer cell for rnn
        if model_cell=='rnn':
            single_cell1=tf.nn.rnn_cell.BasicRNNCell(nh1)
            single_cell2=tf.nn.rnn_cell.BasicRNNCell(nh2)
        elif model_cell=='lstm':
            single_cell1=tf.nn.rnn_cell.BasicLSTMCell(nh1,state_is_tuple=True)
            single_cell2=tf.nn.rnn_cell.BasicLSTMCell(nh2,state_is_tuple=True)
        elif model_cell=='gru':
            single_cell1=tf.nn.rnn_cell.GRUCell(nh1)
            single_cell2=tf.nn.rnn_cell.GRUCell(nh2)
        else:
            raise 'model_cell error!'
        #DropoutWrapper rnn_cell
        single_cell1 = tf.nn.rnn_cell.DropoutWrapper(single_cell1, output_keep_prob=self.keep_prob)
        single_cell2 = tf.nn.rnn_cell.DropoutWrapper(single_cell2, output_keep_prob=self.keep_prob)
      
        self.init_state=single_cell1.zero_state(self.batch_size,dtype=tf.float32) 
        
        #RNN1
        with tf.variable_scope('rnn1'):
            self.outputs1,self.state1=tf.nn.dynamic_rnn(
                cell=single_cell1,
                inputs=inputs,
                initial_state=self.init_state,
                dtype=tf.float32
            )

        #RNN2
        with tf.variable_scope('rnn2'):
            self.outputs2,self.state2=tf.nn.dynamic_rnn(
                cell=single_cell2,
                inputs=self.outputs1,
                initial_state=self.init_state,
                dtype=tf.float32
            )

        #outputs_y
        with tf.variable_scope('output_sy'):
            w_y=tf.get_variable("softmax_w_y",[nh1,ny])
            b_y=tf.get_variable("softmax_b_y",[ny])
            outputs1=tf.reshape(self.outputs1,[-1,nh1])
            sy=tf.nn.xw_plus_b(outputs1,w_y,b_y)
            self.sy_pred = tf.reshape(tf.argmax(sy, 1), [self.batch_size, -1])
        #outputs_z
        with tf.variable_scope('output_sz'):
            w_z = tf.get_variable("softmax_w_z", [nh2, nz])
            b_z = tf.get_variable("softmax_b_z", [nz])
            outputs2 = tf.reshape(self.outputs2, [-1, nh2])
            sz = tf.nn.xw_plus_b(outputs2, w_z,b_z)
            self.sz_pred = tf.reshape(tf.argmax(sz, 1), [self.batch_size, -1])
        #loss
        with tf.variable_scope('loss'):
            label_y = tf.reshape(self.input_y, [-1])
            loss1 = tf.nn.sparse_softmax_cross_entropy_with_logits(sy, label_y)
            label_z = tf.reshape(self.input_z, [-1])
            loss2 = tf.nn.sparse_softmax_cross_entropy_with_logits(sz, label_z)
            self.loss=tf.reduce_sum(0.5*loss1+0.5*loss2)/tf.cast(self.batch_size,tf.float32)

        tvars=tf.trainable_variables()
        grads,_=tf.clip_by_global_norm(tf.gradients(self.loss,tvars),max_gradient_norm)
        optimizer=tf.train.GradientDescentOptimizer(self.lr)
        self.train_op=optimizer.apply_gradients(zip(grads,tvars))

    def cost(output, target):
        # Compute cross entropy for each frame.
        cross_entropy = target * tf.log(output)
        cross_entropy = -tf.reduce_sum(cross_entropy, reduction_indices=2)
        mask = tf.sign(tf.reduce_max(tf.abs(target), reduction_indices=2))
        cross_entropy *= mask
        # Average over actual sequence lengths.
        cross_entropy = tf.reduce_sum(cross_entropy, reduction_indices=1)
        cross_entropy /= tf.reduce_sum(mask, reduction_indices=1)
        return tf.reduce_mean(cross_entropy)

In [ ]:
#load.py


# -*- coding: utf-8 -*-
import numpy as np
import cPickle
import random
def atisfold():
    f = open('data/data_set.pkl')
    train_set, test_set, dicts = cPickle.load(f)
    embedding = cPickle.load(open('data/embedding.pkl'))
    return train_set, test_set,dicts,embedding

def pad_sentences(sentences, padding_word=0, forced_sequence_length=None):
    if forced_sequence_length is None:
        sequence_length=max(len(x) for x in sentences)
    else:
        sequence_length=forced_sequence_length
    padded_sentences=[]
    for i in xrange(len(sentences)):
        sentence=sentences[i]
        num_padding=sequence_length-len(sentence)
        if num_padding<0:
            padded_sentence=sentence[0:sequence_length]
        else:
            padded_sentence=sentence+[int(padding_word)]*num_padding

        padded_sentences.append(padded_sentence)

    return padded_sentences

In [ ]:
#main.py

# -*- coding: utf-8 -*-
import tensorflow as tf
import tensorlayer as tl
import numpy as np
import time
import time
import os
import random
#import load
import models.model as model
import tools
import sys

def main():
    s={
        'nh1':300,
        'nh2':300,
        'win':3,
        'emb_dimension':300,
        'lr':0.1,
        'lr_decay':0.5,
        'max_grad_norm':5,
        'seed':345,
        'nepochs':150,
        'batch_size':16,
        'keep_prob':0.5,
        'check_dir':'./checkpoints',
        'display_test_per':3,
        'lr_decay_per':10
    }

    train_set,test_set,dic,embedding=load.atisfold()
    
    
    idx2label = dict((k,v) for v,k in dic['labels2idx'].iteritems())
    idx2word  = dict((k,v) for v,k in dic['words2idx'].iteritems())

    train_lex, train_y, train_z = train_set

    tr = int(len(train_lex)*0.9)
    valid_lex, valid_y, valid_z = train_lex[tr:], train_y[tr:], train_z[tr:]
    train_lex, train_y, train_z = train_lex[:tr], train_y[:tr], train_z[:tr]
    test_lex,  test_y, test_z  = test_set

    print 'len(train_data) {}'.format(len(train_lex))
    print 'len(valid_data) {}'.format(len(valid_lex))
    print 'len(test_data) {}'.format(len(test_lex))

    vocab = set(dic['words2idx'].keys())
    vocsize = len(vocab)
    print 'len(vocab) {}'.format(vocsize)
    print "Train started!"

    y_nclasses = 2
    z_nclasses = 5

    nsentences = len(train_lex)


    with tf.Session() as sess:
        
        rnn=model.Model(
            nh1=s['nh1'],
            nh2=s['nh2'],
            ny=y_nclasses,
            nz=z_nclasses,
            de=s['emb_dimension'],
            cs=s['win'],
            lr=s['lr'],
            lr_decay=s['lr_decay'],
            embedding=embedding,
            max_gradient_norm=s['max_grad_norm'],
            model_cell='lstm'
        )

        checkpoint_dir=s['check_dir']
        if not os.path.exists(checkpoint_dir):
            os.mkdir(checkpoint_dir)
        checkpoint_prefix=os.path.join(checkpoint_dir,'model')

        def train_step(cwords,label_y,label_z):
            feed={
                rnn.input_x:cwords,
                rnn.input_y:label_y,
                rnn.input_z:label_z,
                rnn.keep_prob:s['keep_prob'],
                rnn.batch_size:s['batch_size']
            }
            fetches=[rnn.loss,rnn.train_op]
            loss,_=sess.run(fetches=fetches,feed_dict=feed)
            return loss

        def dev_step(cwords):
            feed={
                rnn.input_x:cwords,
                rnn.keep_prob:1.0,
                rnn.batch_size:s['batch_size']
            }
            fetches=rnn.sz_pred
            sz_pred=sess.run(fetches=fetches,feed_dict=feed)
            return sz_pred

        saver=tf.train.Saver(tf.all_variables())
        sess.run(tf.initialize_all_variables())

        best_f=-1
        best_e=0
        test_best_f=-1
        test_best_e=0
        best_res=None
        test_best_res=None
        for e in xrange(s['nepochs']):
            tools.shuffle([train_lex,train_y,train_z],s['seed'])
            t_start=time.time()
            for step,batch in enumerate(tl.iterate.minibatches(train_lex,zip(train_y,train_z),batch_size=s['batch_size'])):
                input_x,target=batch
                label_y,label_z=zip(*target)
                input_x=load.pad_sentences(input_x)
                label_y=load.pad_sentences(label_y)
                label_z=load.pad_sentences(label_z)
                cwords=tools.contextwin_2(input_x,s['win'])
                loss=train_step(cwords,label_y,label_z)

                print 'loss %.2f' % loss,' [learning] epoch %i>> %2.2f%%' % (e,s['batch_size']*step*100./nsentences),'completed in %.2f (sec) <<\r' % (time.time()-t_start),

                sys.stdout.flush()

            #VALID

            predictions_valid=[]
            predictions_test=[]
            groundtruth_valid=[]
            groundtruth_test=[]
            for batch in  tl.iterate.minibatches(valid_lex,valid_z,batch_size=s['batch_size']):
                x,z=batch
                x=load.pad_sentences(x)
                x=tools.contextwin_2(x,s['win'])
                predictions_valid.extend(dev_step(x))
                groundtruth_valid.extend(z)

            res_valid=tools.conlleval(predictions_valid,groundtruth_valid,'')

            if res_valid['f']>best_f:
                best_f=res_valid['f']
                best_e=e
                best_res=res_valid
                print '\nVALID new best:',res_valid
                path = saver.save(sess=sess, save_path=checkpoint_prefix, global_step=e)
                print "Save model checkpoint to {}".format(path)
            else:
                print '\nVALID new curr:',res_valid

            #TEST
            if e%s['display_test_per']==0:
                for batch in tl.iterate.minibatches(test_lex, test_z, batch_size=s['batch_size']):
                    x,z = batch
                    x = load.pad_sentences(x)
                    x = tools.contextwin_2(x, s['win'])
                    predictions_test.extend(dev_step(x))
                    groundtruth_test.extend(z)


                res_test = tools.conlleval(predictions_test, groundtruth_test, '')

                if res_test['f'] > test_best_f:
                    test_best_f = res_test['f']
                    test_best_e=e
                    test_best_res=res_test
                    print 'TEST new best:',res_test
                else:
                    print 'TEST new curr:',res_test

            # learning rate decay if no improvement in 10 epochs
            if e-best_e>s['lr_decay_per']:
                sess.run(fetches=rnn.learning_rate_decay_op)
            lr=sess.run(fetches=rnn.lr)
            print 'learning rate:%f' % lr
            if lr<1e-5:break
            print

        print "Train finished!"
        print 'Valid Best Result: epoch %d:  ' % (best_e),best_res
        print 'Test Best Result: epoch %d:  ' %(test_best_e),test_best_res

if __name__ == '__main__':
    main()

In [ ]:
#predict.py

# -*- coding: utf-8 -*-
import tensorflow as tf
import tensorlayer as tl
import numpy as np
import time
import time
import os
import random
import load
import models.model as model

import tools
import sys

def main():
    s={
        'nh1':300,
        'nh2':300,
        'win':3,
        'emb_dimension':300,
        'lr':0.1,
        'lr_decay':0.5,
        'max_grad_norm':5,
        'seed':345,
        'nepochs':50,
        'batch_size':16,
        'keep_prob':1.0,
        'check_dir':'./checkpoints',
        'display_test_per':5,
        'lr_decay_per':10
    }

    
    # load the dataset
    train_set,test_set,dic,embedding=load.atisfold()
    idx2label = dict((k,v) for v,k in dic['labels2idx'].iteritems())
    idx2word  = dict((k,v) for v,k in dic['words2idx'].iteritems())

    vocab = set(dic['words2idx'].keys())
    vocsize = len(vocab)

    test_lex,  test_y, test_z  = test_set[0:1000]

    y_nclasses = 2
    z_nclasses = 5


    with tf.Session() as sess:

        rnn = model.Model(
            nh1=s['nh1'],
            nh2=s['nh2'],
            ny=y_nclasses,
            nz=z_nclasses,
            de=s['emb_dimension'],
            cs=s['win'],
            lr=s['lr'],
            lr_decay=s['lr_decay'],
            embedding=embedding,
            max_gradient_norm=s['max_grad_norm'],
            model_cell='lstm'
        )

        checkpoint_dir = s['check_dir']
        saver = tf.train.Saver(tf.all_variables())
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)

        def dev_step(cwords):
            feed={
                rnn.input_x:cwords,
                rnn.keep_prob:1.0,
                rnn.batch_size:s['batch_size']
            }
            fetches=rnn.sz_pred
            sz_pred=sess.run(fetches=fetches,feed_dict=feed)
            return sz_pred
        print "Test Results : "
        predictions_test=[]
        groundtruth_test=[]
        for batch in tl.iterate.minibatches(test_lex, test_z, batch_size=s['batch_size']):
            x, z = batch
            x = load.pad_sentences(x)
            x = tools.contextwin_2(x, s['win'])
            predictions_test.extend(dev_step(x))
            groundtruth_test.extend(z)

        res_test = tools.conlleval(predictions_test, groundtruth_test, '')

        print res_test

if __name__ == '__main__':
    main()

In [ ]:
#tools.py


# -*- coding: utf-8 -*-
import random

def shuffle(lol,seed):
    '''
    lol :: list of list as input
    seed :: seed the shuffling
    shuffle inplace each list in the same order
    '''
    for l in lol:
        random.seed(seed)
        random.shuffle(l)

def contextwin(l, win):
    '''
    win :: int corresponding to the size of the window
    given a list of indexes composing a sentence
    it will return a list of list of indexes corresponding
    to context windows surrounding each word in the sentence
    '''
    assert (win % 2) == 1
    assert win >=1
    l = list(l)

    lpadded = win/2 * [0] + l + win/2 * [0]
    out = [ lpadded[i:i+win] for i in range(len(l)) ]

    assert len(out) == len(l)
    return out

def contextwin_2(ls,win):
    assert (win % 2) == 1
    assert win >=1
    outs=[]
    for l in ls:
        outs.append(contextwin(l,win))
    return outs

def getKeyphraseList(l):
    res, now= [], []
    for i in xrange(len(l)):
        if l[i] != 0:
            now.append(str(i))
        if l[i] == 0 or i == len(l) - 1:
            if len(now) != 0:
                res.append(' '.join(now))
            now = []
    return set(res)

def conlleval(predictions, groundtruth, file):
    assert len(predictions) == len(groundtruth)
    res = {}
    all_cnt, good_cnt = len(predictions), 0
    p_cnt, r_cnt, pr_cnt = 0, 0, 0
    for i in range(all_cnt):
        # print i
        if all(predictions[i][0:len(groundtruth[i])] == groundtruth[i]) == True:
            good_cnt += 1
        pKeyphraseList = getKeyphraseList(predictions[i][0:len(groundtruth[i])])
        gKeyphraseList = getKeyphraseList(groundtruth[i])
        p_cnt += len(pKeyphraseList)
        r_cnt += len(gKeyphraseList)
        pr_cnt += len(pKeyphraseList & gKeyphraseList)
    res['a'] = 1.0*good_cnt/all_cnt
    res['p'] = 1.0*good_cnt/p_cnt
    res['r'] = 1.0*good_cnt/r_cnt
    res['f'] = 2.0*res['p']*res['r']/(res['p']+res['r'])
    return res

In [2]:
import tensorflow as tf